# Connect to DB and review tables

In [1]:
import os
from dotenv import load_dotenv
import mysql.connector
import requests
import pandas as pd

# Load environment variables from .env file
load_dotenv()

host=os.getenv("MYSQL_HOST")
user=os.getenv("MYSQL_USER")
password=os.getenv("MYSQL_PASSWORD")
database=os.getenv("MYSQL_DB")
port=os.getenv("MYSQL_PORT")


# Create connection
db = mysql.connector.connect(
  host=host,
  user=user,
  password=password,
  database=database,
  port=port
)

In [2]:
# Show tables
cursor = db.cursor()
cursor.execute("SHOW TABLES")
cursor.fetchall()

[('customer',), ('product',), ('transaction',)]

In [3]:
# Describe tables
tables = ["customer", "product", "transaction"]
for table in tables:
    cursor.execute(f"DESCRIBE {table}")
    result = cursor.fetchall()
    print(f"-- {table} --")
    print(result)
    print()

-- customer --
[('CustomerNo', 'double', 'YES', '', None, ''), ('Country', 'text', 'YES', '', None, ''), ('Name', 'text', 'YES', '', None, '')]

-- product --
[('ProductNo', 'text', 'YES', '', None, ''), ('ProductName', 'text', 'YES', '', None, '')]

-- transaction --
[('TransactionNo', 'text', 'YES', '', None, ''), ('Date', 'datetime', 'YES', '', None, ''), ('ProductNo', 'text', 'YES', '', None, ''), ('Price', 'double', 'YES', '', None, ''), ('Quantity', 'bigint(20)', 'YES', '', None, ''), ('CustomerNo', 'double', 'YES', '', None, '')]



# Convert to Pandas DataFrame

In [4]:
import pandas as pd

def to_df(table):
    cursor.execute(f"SELECT * FROM {table}")
    result = cursor.fetchall()
    columns = [col[0] for col in cursor.description] # To also show header
    return pd.DataFrame(result, columns=columns)

product = to_df("product")
customer = to_df("customer")
transaction = to_df("transaction")

In [5]:
product

,ProductNo,ProductName
0,22485,Set Of 2 Wooden Market Crates
1,22596,Christmas Star Wish List Chalkboard
2,23235,Storage Tin Vintage Leaf
3,23272,Tree T-Light Holder Willie Winkie
4,23239,Set Of 4 Knick Knack Tins Poppies
...,...,...
3763,22275,Weekend Bag Vintage Rose Paisley
3764,16161M,Wrap Pink Flock
3765,84854,Girly Pink Tool Set
3766,82615,Pink Marshmallow Scarf Knitting Kit


In [6]:
customer

,CustomerNo,Country,Name
0,17490.0,United Kingdom,Sara Griffin
1,13069.0,United Kingdom,Michael Holt
2,12433.0,Norway,Kelli Sandoval
3,13426.0,United Kingdom,Dalton Graves
4,17364.0,United Kingdom,Michelle James
...,...,...,...
4734,16274.0,United Kingdom,Megan Young
4735,14142.0,United Kingdom,Luke Williams
4736,13065.0,United Kingdom,Lisa Jones
4737,18011.0,United Kingdom,Kelly Jenkins


In [7]:
transaction

,TransactionNo,Date,ProductNo,Price,Quantity,CustomerNo
0,581482,2024-05-10,22485,21.47,12,17490.0
1,581475,2024-05-10,22596,10.65,36,13069.0
2,581475,2024-05-10,23235,11.53,12,13069.0
3,581475,2024-05-10,23272,10.65,12,13069.0
4,581475,2024-05-10,23239,11.94,6,13069.0
...,...,...,...,...,...,...
536345,C536548,2023-05-03,22168,18.96,-2,12472.0
536346,C536548,2023-05-03,21218,14.09,-3,12472.0
536347,C536548,2023-05-03,20957,11.74,-1,12472.0
536348,C536548,2023-05-03,22580,16.35,-4,12472.0


# Collect data using API and convert to DataFrame

In [8]:
import requests

# Get data from API
def api():
    url = "https://r2de3-currency-api-vmftiryt6q-as.a.run.app/gbp_thb"
    r = requests.get(url)
    result = r.json()
    return pd.DataFrame(result)

conversion_rate = api()

In [9]:
api()

,date,gbp_thb,id
0,2023-05-01,42.761,37af
1,2023-05-02,42.477,c86f
2,2023-05-03,42.630,f3c6
3,2023-05-04,42.456,3f59
4,2023-05-05,42.794,c3e3
...,...,...,...
384,2024-05-19,45.957,a67d
385,2024-05-20,45.859,69aa
386,2024-05-21,46.246,975c
387,2024-05-22,46.382,8108


# Merge tables

In [10]:
# Check data type
df_tables = [product, customer, transaction]
for table in df_tables:
    print(table.dtypes)
    print()

ProductNo      object
ProductName    object
dtype: object

CustomerNo    float64
Country        object
Name           object
dtype: object

TransactionNo            object
Date             datetime64[ns]
ProductNo                object
Price                   float64
Quantity                  int64
CustomerNo              float64
dtype: object



In [11]:
# Check rows
transaction.describe()

,Date,Price,Quantity,CustomerNo
count,536350,536350.000000,536350.000000,536295.000000
mean,2023-12-04 02:52:31.891116032,12.662182,9.919347,15227.893178
min,2023-05-03 00:00:00,5.130000,-80995.000000,12004.000000
25%,2023-08-28 00:00:00,10.990000,1.000000,13807.000000
50%,2023-12-20 00:00:00,11.940000,3.000000,15152.000000
75%,2024-03-20 00:00:00,14.090000,10.000000,16729.000000
max,2024-05-10 00:00:00,660.620000,80995.000000,18287.000000
std,NaN,8.490450,216.662300,1716.582932


In [12]:
# Merge product, customer, and transaction
merged_df = transaction.merge(product, how='left', on='ProductNo').merge(customer, how='left', on='CustomerNo')
merged_df

,TransactionNo,Date,ProductNo,Price,Quantity,CustomerNo,ProductName,Country,Name
0,581482,2024-05-10,22485,21.47,12,17490.0,Set Of 2 Wooden Market Crates,United Kingdom,Sara Griffin
1,581475,2024-05-10,22596,10.65,36,13069.0,Christmas Star Wish List Chalkboard,United Kingdom,Michael Holt
2,581475,2024-05-10,23235,11.53,12,13069.0,Storage Tin Vintage Leaf,United Kingdom,Michael Holt
3,581475,2024-05-10,23272,10.65,12,13069.0,Tree T-Light Holder Willie Winkie,United Kingdom,Michael Holt
4,581475,2024-05-10,23239,11.94,6,13069.0,Set Of 4 Knick Knack Tins Poppies,United Kingdom,Michael Holt
...,...,...,...,...,...,...,...,...,...
536345,C536548,2023-05-03,22168,18.96,-2,12472.0,Organiser Wood Antique White,Germany,Stephen Pena
536346,C536548,2023-05-03,21218,14.09,-3,12472.0,Red Spotty Biscuit Tin,Germany,Stephen Pena
536347,C536548,2023-05-03,20957,11.74,-1,12472.0,Porcelain Hanging Bell Small,Germany,Stephen Pena
536348,C536548,2023-05-03,22580,16.35,-4,12472.0,Advent Calendar Gingham Sack,Germany,Stephen Pena


In [13]:
# Check conversion_rate data type
conversion_rate.dtypes

date        object
gbp_thb    float64
id          object
dtype: object

In [14]:
# Check rows
conversion_rate.describe()

,gbp_thb
count,389.000000
mean,44.620663
std,0.956360
min,42.267000
25%,44.050000
50%,44.523000
75%,45.366000
max,46.504000


In [15]:
# Check format
conversion_rate[conversion_rate['date'].str.contains(r"^\d{4}\-\d{2}\-\d{2}$")]

,date,gbp_thb,id
0,2023-05-01,42.761,37af
1,2023-05-02,42.477,c86f
2,2023-05-03,42.630,f3c6
3,2023-05-04,42.456,3f59
4,2023-05-05,42.794,c3e3
...,...,...,...
384,2024-05-19,45.957,a67d
385,2024-05-20,45.859,69aa
386,2024-05-21,46.246,975c
387,2024-05-22,46.382,8108


In [16]:
# Cast to date-data-type
conversion_rate['date'] = pd.to_datetime(conversion_rate['date'])

In [17]:
# Recheck data type
conversion_rate.dtypes

date       datetime64[ns]
gbp_thb           float64
id                 object
dtype: object

In [18]:
# Merge merged_df, conversion_rate as final_df
final_df = merged_df.merge(conversion_rate, how='left', left_on='Date', right_on='date')
final_df

,TransactionNo,Date,ProductNo,Price,Quantity,CustomerNo,ProductName,Country,Name,date,gbp_thb,id
0,581482,2024-05-10,22485,21.47,12,17490.0,Set Of 2 Wooden Market Crates,United Kingdom,Sara Griffin,2024-05-10,45.77,a19b
1,581475,2024-05-10,22596,10.65,36,13069.0,Christmas Star Wish List Chalkboard,United Kingdom,Michael Holt,2024-05-10,45.77,a19b
2,581475,2024-05-10,23235,11.53,12,13069.0,Storage Tin Vintage Leaf,United Kingdom,Michael Holt,2024-05-10,45.77,a19b
3,581475,2024-05-10,23272,10.65,12,13069.0,Tree T-Light Holder Willie Winkie,United Kingdom,Michael Holt,2024-05-10,45.77,a19b
4,581475,2024-05-10,23239,11.94,6,13069.0,Set Of 4 Knick Knack Tins Poppies,United Kingdom,Michael Holt,2024-05-10,45.77,a19b
...,...,...,...,...,...,...,...,...,...,...,...,...
536345,C536548,2023-05-03,22168,18.96,-2,12472.0,Organiser Wood Antique White,Germany,Stephen Pena,2023-05-03,42.63,f3c6
536346,C536548,2023-05-03,21218,14.09,-3,12472.0,Red Spotty Biscuit Tin,Germany,Stephen Pena,2023-05-03,42.63,f3c6
536347,C536548,2023-05-03,20957,11.74,-1,12472.0,Porcelain Hanging Bell Small,Germany,Stephen Pena,2023-05-03,42.63,f3c6
536348,C536548,2023-05-03,22580,16.35,-4,12472.0,Advent Calendar Gingham Sack,Germany,Stephen Pena,2023-05-03,42.63,f3c6


# Clean final_df

### Drop duplicates

In [19]:
final_df.duplicated().sum()

# 5200 rows

5200

In [20]:
final_df = final_df.drop_duplicates()
final_df.duplicated().sum()

0

In [21]:
final_df

# Before 536350
# After 536350 - 5200 = 531150

,TransactionNo,Date,ProductNo,Price,Quantity,CustomerNo,ProductName,Country,Name,date,gbp_thb,id
0,581482,2024-05-10,22485,21.47,12,17490.0,Set Of 2 Wooden Market Crates,United Kingdom,Sara Griffin,2024-05-10,45.77,a19b
1,581475,2024-05-10,22596,10.65,36,13069.0,Christmas Star Wish List Chalkboard,United Kingdom,Michael Holt,2024-05-10,45.77,a19b
2,581475,2024-05-10,23235,11.53,12,13069.0,Storage Tin Vintage Leaf,United Kingdom,Michael Holt,2024-05-10,45.77,a19b
3,581475,2024-05-10,23272,10.65,12,13069.0,Tree T-Light Holder Willie Winkie,United Kingdom,Michael Holt,2024-05-10,45.77,a19b
4,581475,2024-05-10,23239,11.94,6,13069.0,Set Of 4 Knick Knack Tins Poppies,United Kingdom,Michael Holt,2024-05-10,45.77,a19b
...,...,...,...,...,...,...,...,...,...,...,...,...
536345,C536548,2023-05-03,22168,18.96,-2,12472.0,Organiser Wood Antique White,Germany,Stephen Pena,2023-05-03,42.63,f3c6
536346,C536548,2023-05-03,21218,14.09,-3,12472.0,Red Spotty Biscuit Tin,Germany,Stephen Pena,2023-05-03,42.63,f3c6
536347,C536548,2023-05-03,20957,11.74,-1,12472.0,Porcelain Hanging Bell Small,Germany,Stephen Pena,2023-05-03,42.63,f3c6
536348,C536548,2023-05-03,22580,16.35,-4,12472.0,Advent Calendar Gingham Sack,Germany,Stephen Pena,2023-05-03,42.63,f3c6


### Check null values

In [22]:
final_df.isna().sum()

TransactionNo     0
Date              0
ProductNo         0
Price             0
Quantity          0
CustomerNo       55
ProductName       0
Country           0
Name              0
date              0
gbp_thb           0
id                0
dtype: int64

In [23]:
final_df[final_df['CustomerNo'].isna()]

,TransactionNo,Date,ProductNo,Price,Quantity,CustomerNo,ProductName,Country,Name,date,gbp_thb,id
6511,C581406,2024-05-09,46000M,6.19,-240,NaN,Polyester Filler Pad 45x45cm,United Kingdom,Allen Morgan,2024-05-09,46.040,7619
6512,C581406,2024-05-09,46000S,6.19,-300,NaN,Polyester Filler Pad 40x40cm,United Kingdom,Allen Morgan,2024-05-09,46.040,7619
90098,C575153,2024-04-09,22947,44.25,-1,NaN,Wooden Advent Calendar Red,United Kingdom,Allen Morgan,2024-04-09,46.022,db11
102671,C574288,2024-04-04,22178,25.37,-1,NaN,Victorian Glass Hanging T-Light,United Kingdom,Allen Morgan,2024-04-04,46.402,cde9
117263,C573180,2024-03-29,23048,14.50,-1,NaN,Set Of 10 Lanterns Fairy Light Star,United Kingdom,Allen Morgan,2024-03-29,45.894,18a7
163160,C569495,2024-03-05,21843,21.47,-1,NaN,Red Retrospot Cake Stand,United Kingdom,Allen Morgan,2024-03-05,45.477,4388
190598,C567518,2024-02-20,22846,27.62,-1,NaN,Bread Bin Diner Style Red,United Kingdom,Allen Morgan,2024-02-20,45.456,f1ca
192284,C567518,2024-02-20,21871,11.94,-12,NaN,Save The Planet Mug,United Kingdom,Allen Morgan,2024-02-20,45.456,f1ca
242406,C563015,2024-01-11,46000M,10.25,-160,NaN,Polyester Filler Pad 45x45cm,United Kingdom,Allen Morgan,2024-01-11,44.736,1045
242407,C563015,2024-01-11,46000S,10.25,-220,NaN,Polyester Filler Pad 40x40cm,United Kingdom,Allen Morgan,2024-01-11,44.736,1045


In [24]:
final_df = final_df.dropna()
final_df.isna().sum()

TransactionNo    0
Date             0
ProductNo        0
Price            0
Quantity         0
CustomerNo       0
ProductName      0
Country          0
Name             0
date             0
gbp_thb          0
id               0
dtype: int64

In [25]:
final_df

# Before 531150
# After 531150 - 55 = 531095

,TransactionNo,Date,ProductNo,Price,Quantity,CustomerNo,ProductName,Country,Name,date,gbp_thb,id
0,581482,2024-05-10,22485,21.47,12,17490.0,Set Of 2 Wooden Market Crates,United Kingdom,Sara Griffin,2024-05-10,45.77,a19b
1,581475,2024-05-10,22596,10.65,36,13069.0,Christmas Star Wish List Chalkboard,United Kingdom,Michael Holt,2024-05-10,45.77,a19b
2,581475,2024-05-10,23235,11.53,12,13069.0,Storage Tin Vintage Leaf,United Kingdom,Michael Holt,2024-05-10,45.77,a19b
3,581475,2024-05-10,23272,10.65,12,13069.0,Tree T-Light Holder Willie Winkie,United Kingdom,Michael Holt,2024-05-10,45.77,a19b
4,581475,2024-05-10,23239,11.94,6,13069.0,Set Of 4 Knick Knack Tins Poppies,United Kingdom,Michael Holt,2024-05-10,45.77,a19b
...,...,...,...,...,...,...,...,...,...,...,...,...
536345,C536548,2023-05-03,22168,18.96,-2,12472.0,Organiser Wood Antique White,Germany,Stephen Pena,2023-05-03,42.63,f3c6
536346,C536548,2023-05-03,21218,14.09,-3,12472.0,Red Spotty Biscuit Tin,Germany,Stephen Pena,2023-05-03,42.63,f3c6
536347,C536548,2023-05-03,20957,11.74,-1,12472.0,Porcelain Hanging Bell Small,Germany,Stephen Pena,2023-05-03,42.63,f3c6
536348,C536548,2023-05-03,22580,16.35,-4,12472.0,Advent Calendar Gingham Sack,Germany,Stephen Pena,2023-05-03,42.63,f3c6


### TransactionNo column

##### Check format

In [26]:
# Six-number format
final_df[final_df['TransactionNo'].str.match(r"^\d{6}$")]

,TransactionNo,Date,ProductNo,Price,Quantity,CustomerNo,ProductName,Country,Name,date,gbp_thb,id
0,581482,2024-05-10,22485,21.47,12,17490.0,Set Of 2 Wooden Market Crates,United Kingdom,Sara Griffin,2024-05-10,45.77,a19b
1,581475,2024-05-10,22596,10.65,36,13069.0,Christmas Star Wish List Chalkboard,United Kingdom,Michael Holt,2024-05-10,45.77,a19b
2,581475,2024-05-10,23235,11.53,12,13069.0,Storage Tin Vintage Leaf,United Kingdom,Michael Holt,2024-05-10,45.77,a19b
3,581475,2024-05-10,23272,10.65,12,13069.0,Tree T-Light Holder Willie Winkie,United Kingdom,Michael Holt,2024-05-10,45.77,a19b
4,581475,2024-05-10,23239,11.94,6,13069.0,Set Of 4 Knick Knack Tins Poppies,United Kingdom,Michael Holt,2024-05-10,45.77,a19b
...,...,...,...,...,...,...,...,...,...,...,...,...
536320,536585,2023-05-03,37449,20.45,2,17460.0,Ceramic Cake Stand + Hanging Cakes,United Kingdom,Megan Schmitt,2023-05-03,42.63,f3c6
536321,536590,2023-05-03,22776,20.45,1,13065.0,Sweetheart 3 Tier Cake Stand,United Kingdom,Lisa Jones,2023-05-03,42.63,f3c6
536322,536590,2023-05-03,22622,20.45,2,13065.0,Box Of Vintage Alphabet Blocks,United Kingdom,Lisa Jones,2023-05-03,42.63,f3c6
536323,536591,2023-05-03,37449,20.45,1,14606.0,Ceramic Cake Stand + Hanging Cakes,United Kingdom,Matthew Wells,2023-05-03,42.63,f3c6


In [27]:
# Other format
final_df[~final_df['TransactionNo'].str.match(r"^\d{6}$")] # 8494 rows

,TransactionNo,Date,ProductNo,Price,Quantity,CustomerNo,ProductName,Country,Name,date,gbp_thb,id
1616,C581484,2024-05-10,23843,6.19,-80995,16446.0,Paper Craft Little Birdie,United Kingdom,John Burns,2024-05-10,45.77,a19b
1617,C581490,2024-05-10,22178,6.19,-12,14397.0,Victorian Glass Hanging T-Light,United Kingdom,Richard Kelly,2024-05-10,45.77,a19b
1618,C581490,2024-05-10,23144,6.04,-11,14397.0,Zinc T-Light Holder Stars Small,United Kingdom,Richard Kelly,2024-05-10,45.77,a19b
1619,C581568,2024-05-10,21258,6.19,-5,15311.0,Victorian Sewing Box Large,United Kingdom,Charles Swanson,2024-05-10,45.77,a19b
1620,C581569,2024-05-10,84978,6.19,-1,17315.0,Hanging Heart Jar T-Light Holder,United Kingdom,Casey Davis,2024-05-10,45.77,a19b
...,...,...,...,...,...,...,...,...,...,...,...,...
536345,C536548,2023-05-03,22168,18.96,-2,12472.0,Organiser Wood Antique White,Germany,Stephen Pena,2023-05-03,42.63,f3c6
536346,C536548,2023-05-03,21218,14.09,-3,12472.0,Red Spotty Biscuit Tin,Germany,Stephen Pena,2023-05-03,42.63,f3c6
536347,C536548,2023-05-03,20957,11.74,-1,12472.0,Porcelain Hanging Bell Small,Germany,Stephen Pena,2023-05-03,42.63,f3c6
536348,C536548,2023-05-03,22580,16.35,-4,12472.0,Advent Calendar Gingham Sack,Germany,Stephen Pena,2023-05-03,42.63,f3c6


In [28]:
# Check the number of rows of C... format 
start_with_C = final_df[final_df['TransactionNo'].str.match(r"^[C]\d{6}$")]
start_with_C 

# Other format = 8494 rows
# start_with_C format = 8494 rows

,TransactionNo,Date,ProductNo,Price,Quantity,CustomerNo,ProductName,Country,Name,date,gbp_thb,id
1616,C581484,2024-05-10,23843,6.19,-80995,16446.0,Paper Craft Little Birdie,United Kingdom,John Burns,2024-05-10,45.77,a19b
1617,C581490,2024-05-10,22178,6.19,-12,14397.0,Victorian Glass Hanging T-Light,United Kingdom,Richard Kelly,2024-05-10,45.77,a19b
1618,C581490,2024-05-10,23144,6.04,-11,14397.0,Zinc T-Light Holder Stars Small,United Kingdom,Richard Kelly,2024-05-10,45.77,a19b
1619,C581568,2024-05-10,21258,6.19,-5,15311.0,Victorian Sewing Box Large,United Kingdom,Charles Swanson,2024-05-10,45.77,a19b
1620,C581569,2024-05-10,84978,6.19,-1,17315.0,Hanging Heart Jar T-Light Holder,United Kingdom,Casey Davis,2024-05-10,45.77,a19b
...,...,...,...,...,...,...,...,...,...,...,...,...
536345,C536548,2023-05-03,22168,18.96,-2,12472.0,Organiser Wood Antique White,Germany,Stephen Pena,2023-05-03,42.63,f3c6
536346,C536548,2023-05-03,21218,14.09,-3,12472.0,Red Spotty Biscuit Tin,Germany,Stephen Pena,2023-05-03,42.63,f3c6
536347,C536548,2023-05-03,20957,11.74,-1,12472.0,Porcelain Hanging Bell Small,Germany,Stephen Pena,2023-05-03,42.63,f3c6
536348,C536548,2023-05-03,22580,16.35,-4,12472.0,Advent Calendar Gingham Sack,Germany,Stephen Pena,2023-05-03,42.63,f3c6


##### Create a mock table to check what TransactionNo starting with C is

In [29]:
mock_df = final_df[final_df['TransactionNo'].str.match(r"^[C]\d{6}$")]
mock_df

,TransactionNo,Date,ProductNo,Price,Quantity,CustomerNo,ProductName,Country,Name,date,gbp_thb,id
1616,C581484,2024-05-10,23843,6.19,-80995,16446.0,Paper Craft Little Birdie,United Kingdom,John Burns,2024-05-10,45.77,a19b
1617,C581490,2024-05-10,22178,6.19,-12,14397.0,Victorian Glass Hanging T-Light,United Kingdom,Richard Kelly,2024-05-10,45.77,a19b
1618,C581490,2024-05-10,23144,6.04,-11,14397.0,Zinc T-Light Holder Stars Small,United Kingdom,Richard Kelly,2024-05-10,45.77,a19b
1619,C581568,2024-05-10,21258,6.19,-5,15311.0,Victorian Sewing Box Large,United Kingdom,Charles Swanson,2024-05-10,45.77,a19b
1620,C581569,2024-05-10,84978,6.19,-1,17315.0,Hanging Heart Jar T-Light Holder,United Kingdom,Casey Davis,2024-05-10,45.77,a19b
...,...,...,...,...,...,...,...,...,...,...,...,...
536345,C536548,2023-05-03,22168,18.96,-2,12472.0,Organiser Wood Antique White,Germany,Stephen Pena,2023-05-03,42.63,f3c6
536346,C536548,2023-05-03,21218,14.09,-3,12472.0,Red Spotty Biscuit Tin,Germany,Stephen Pena,2023-05-03,42.63,f3c6
536347,C536548,2023-05-03,20957,11.74,-1,12472.0,Porcelain Hanging Bell Small,Germany,Stephen Pena,2023-05-03,42.63,f3c6
536348,C536548,2023-05-03,22580,16.35,-4,12472.0,Advent Calendar Gingham Sack,Germany,Stephen Pena,2023-05-03,42.63,f3c6


In [30]:
# The Quantity column is a negative number
# Cast to string data type to check Quantity column
mock_df['Quantity'] = mock_df['Quantity'].astype(str)

/var/folders/0_/s8sxzn9n5jq5cx7f64dbfmj40000gn/T/ipykernel_3383/774255657.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  mock_df['Quantity'] = mock_df['Quantity'].astype(str)


In [31]:
mock_df.dtypes

TransactionNo            object
Date             datetime64[ns]
ProductNo                object
Price                   float64
Quantity                 object
CustomerNo              float64
ProductName              object
Country                  object
Name                     object
date             datetime64[ns]
gbp_thb                 float64
id                       object
dtype: object

In [32]:
mock_df[mock_df['Quantity'].str.contains(r"^\-\d+$")] 

# All rows in Quantity column are a negative number. So, assuming that it is a product return. I'll leave it like that.

,TransactionNo,Date,ProductNo,Price,Quantity,CustomerNo,ProductName,Country,Name,date,gbp_thb,id
1616,C581484,2024-05-10,23843,6.19,-80995,16446.0,Paper Craft Little Birdie,United Kingdom,John Burns,2024-05-10,45.77,a19b
1617,C581490,2024-05-10,22178,6.19,-12,14397.0,Victorian Glass Hanging T-Light,United Kingdom,Richard Kelly,2024-05-10,45.77,a19b
1618,C581490,2024-05-10,23144,6.04,-11,14397.0,Zinc T-Light Holder Stars Small,United Kingdom,Richard Kelly,2024-05-10,45.77,a19b
1619,C581568,2024-05-10,21258,6.19,-5,15311.0,Victorian Sewing Box Large,United Kingdom,Charles Swanson,2024-05-10,45.77,a19b
1620,C581569,2024-05-10,84978,6.19,-1,17315.0,Hanging Heart Jar T-Light Holder,United Kingdom,Casey Davis,2024-05-10,45.77,a19b
...,...,...,...,...,...,...,...,...,...,...,...,...
536345,C536548,2023-05-03,22168,18.96,-2,12472.0,Organiser Wood Antique White,Germany,Stephen Pena,2023-05-03,42.63,f3c6
536346,C536548,2023-05-03,21218,14.09,-3,12472.0,Red Spotty Biscuit Tin,Germany,Stephen Pena,2023-05-03,42.63,f3c6
536347,C536548,2023-05-03,20957,11.74,-1,12472.0,Porcelain Hanging Bell Small,Germany,Stephen Pena,2023-05-03,42.63,f3c6
536348,C536548,2023-05-03,22580,16.35,-4,12472.0,Advent Calendar Gingham Sack,Germany,Stephen Pena,2023-05-03,42.63,f3c6


### CustomerNo column

In [33]:
# Cast type to String data type to match pattern
final_df['CustomerNo'] = final_df['CustomerNo'].astype(str)
final_df.dtypes

/var/folders/0_/s8sxzn9n5jq5cx7f64dbfmj40000gn/T/ipykernel_3383/972426763.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  final_df['CustomerNo'] = final_df['CustomerNo'].astype(str)


TransactionNo            object
Date             datetime64[ns]
ProductNo                object
Price                   float64
Quantity                  int64
CustomerNo               object
ProductName              object
Country                  object
Name                     object
date             datetime64[ns]
gbp_thb                 float64
id                       object
dtype: object

In [34]:
# Match pattern
final_df[final_df['CustomerNo'].str.match(r"^\d{5}\.[0]$")]

,TransactionNo,Date,ProductNo,Price,Quantity,CustomerNo,ProductName,Country,Name,date,gbp_thb,id
0,581482,2024-05-10,22485,21.47,12,17490.0,Set Of 2 Wooden Market Crates,United Kingdom,Sara Griffin,2024-05-10,45.77,a19b
1,581475,2024-05-10,22596,10.65,36,13069.0,Christmas Star Wish List Chalkboard,United Kingdom,Michael Holt,2024-05-10,45.77,a19b
2,581475,2024-05-10,23235,11.53,12,13069.0,Storage Tin Vintage Leaf,United Kingdom,Michael Holt,2024-05-10,45.77,a19b
3,581475,2024-05-10,23272,10.65,12,13069.0,Tree T-Light Holder Willie Winkie,United Kingdom,Michael Holt,2024-05-10,45.77,a19b
4,581475,2024-05-10,23239,11.94,6,13069.0,Set Of 4 Knick Knack Tins Poppies,United Kingdom,Michael Holt,2024-05-10,45.77,a19b
...,...,...,...,...,...,...,...,...,...,...,...,...
536345,C536548,2023-05-03,22168,18.96,-2,12472.0,Organiser Wood Antique White,Germany,Stephen Pena,2023-05-03,42.63,f3c6
536346,C536548,2023-05-03,21218,14.09,-3,12472.0,Red Spotty Biscuit Tin,Germany,Stephen Pena,2023-05-03,42.63,f3c6
536347,C536548,2023-05-03,20957,11.74,-1,12472.0,Porcelain Hanging Bell Small,Germany,Stephen Pena,2023-05-03,42.63,f3c6
536348,C536548,2023-05-03,22580,16.35,-4,12472.0,Advent Calendar Gingham Sack,Germany,Stephen Pena,2023-05-03,42.63,f3c6


In [35]:
# Cast type to Integer
final_df['CustomerNo'] = final_df['CustomerNo'].str[:5].astype(int)
final_df

/var/folders/0_/s8sxzn9n5jq5cx7f64dbfmj40000gn/T/ipykernel_3383/3891840535.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  final_df['CustomerNo'] = final_df['CustomerNo'].str[:5].astype(int)


,TransactionNo,Date,ProductNo,Price,Quantity,CustomerNo,ProductName,Country,Name,date,gbp_thb,id
0,581482,2024-05-10,22485,21.47,12,17490,Set Of 2 Wooden Market Crates,United Kingdom,Sara Griffin,2024-05-10,45.77,a19b
1,581475,2024-05-10,22596,10.65,36,13069,Christmas Star Wish List Chalkboard,United Kingdom,Michael Holt,2024-05-10,45.77,a19b
2,581475,2024-05-10,23235,11.53,12,13069,Storage Tin Vintage Leaf,United Kingdom,Michael Holt,2024-05-10,45.77,a19b
3,581475,2024-05-10,23272,10.65,12,13069,Tree T-Light Holder Willie Winkie,United Kingdom,Michael Holt,2024-05-10,45.77,a19b
4,581475,2024-05-10,23239,11.94,6,13069,Set Of 4 Knick Knack Tins Poppies,United Kingdom,Michael Holt,2024-05-10,45.77,a19b
...,...,...,...,...,...,...,...,...,...,...,...,...
536345,C536548,2023-05-03,22168,18.96,-2,12472,Organiser Wood Antique White,Germany,Stephen Pena,2023-05-03,42.63,f3c6
536346,C536548,2023-05-03,21218,14.09,-3,12472,Red Spotty Biscuit Tin,Germany,Stephen Pena,2023-05-03,42.63,f3c6
536347,C536548,2023-05-03,20957,11.74,-1,12472,Porcelain Hanging Bell Small,Germany,Stephen Pena,2023-05-03,42.63,f3c6
536348,C536548,2023-05-03,22580,16.35,-4,12472,Advent Calendar Gingham Sack,Germany,Stephen Pena,2023-05-03,42.63,f3c6


In [36]:
final_df.dtypes

TransactionNo            object
Date             datetime64[ns]
ProductNo                object
Price                   float64
Quantity                  int64
CustomerNo                int64
ProductName              object
Country                  object
Name                     object
date             datetime64[ns]
gbp_thb                 float64
id                       object
dtype: object

### Product column

In [37]:
# Five-number-format
final_df[final_df['ProductNo'].str.match(r"^\d{5}$")]

# 480715 rows

,TransactionNo,Date,ProductNo,Price,Quantity,CustomerNo,ProductName,Country,Name,date,gbp_thb,id
0,581482,2024-05-10,22485,21.47,12,17490,Set Of 2 Wooden Market Crates,United Kingdom,Sara Griffin,2024-05-10,45.77,a19b
1,581475,2024-05-10,22596,10.65,36,13069,Christmas Star Wish List Chalkboard,United Kingdom,Michael Holt,2024-05-10,45.77,a19b
2,581475,2024-05-10,23235,11.53,12,13069,Storage Tin Vintage Leaf,United Kingdom,Michael Holt,2024-05-10,45.77,a19b
3,581475,2024-05-10,23272,10.65,12,13069,Tree T-Light Holder Willie Winkie,United Kingdom,Michael Holt,2024-05-10,45.77,a19b
4,581475,2024-05-10,23239,11.94,6,13069,Set Of 4 Knick Knack Tins Poppies,United Kingdom,Michael Holt,2024-05-10,45.77,a19b
...,...,...,...,...,...,...,...,...,...,...,...,...
536345,C536548,2023-05-03,22168,18.96,-2,12472,Organiser Wood Antique White,Germany,Stephen Pena,2023-05-03,42.63,f3c6
536346,C536548,2023-05-03,21218,14.09,-3,12472,Red Spotty Biscuit Tin,Germany,Stephen Pena,2023-05-03,42.63,f3c6
536347,C536548,2023-05-03,20957,11.74,-1,12472,Porcelain Hanging Bell Small,Germany,Stephen Pena,2023-05-03,42.63,f3c6
536348,C536548,2023-05-03,22580,16.35,-4,12472,Advent Calendar Gingham Sack,Germany,Stephen Pena,2023-05-03,42.63,f3c6


In [38]:
# Other format
final_df[~final_df['ProductNo'].str.match(r"^\d{5}$")]

# 50380 rows

,TransactionNo,Date,ProductNo,Price,Quantity,CustomerNo,ProductName,Country,Name,date,gbp_thb,id
68,581476,2024-05-10,84596F,10.68,32,12433,Small Marshmallows Pink Bowl,Norway,Kelli Sandoval,2024-05-10,45.77,a19b
69,581476,2024-05-10,84596B,10.68,16,12433,Small Dolly Mix Design Orange Bowl,Norway,Kelli Sandoval,2024-05-10,45.77,a19b
70,581476,2024-05-10,84510A,11.53,20,12433,Set Of 4 English Rose Coasters,Norway,Kelli Sandoval,2024-05-10,45.77,a19b
74,581476,2024-05-10,47559B,11.53,10,12433,Tea Time Oven Glove,Norway,Kelli Sandoval,2024-05-10,45.77,a19b
75,581476,2024-05-10,47504H,11.06,36,12433,English Rose Spirit Level,Norway,Kelli Sandoval,2024-05-10,45.77,a19b
...,...,...,...,...,...,...,...,...,...,...,...,...
536272,536592,2023-05-03,47590B,21.55,1,16592,Pink Happy Birthday Bunting,United Kingdom,Elizabeth Wong,2023-05-03,42.63,f3c6
536279,536401,2023-05-03,47570B,21.17,1,15862,Tea Time Table Cloth,United Kingdom,Jonathan Lee,2023-05-03,42.63,f3c6
536289,536592,2023-05-03,85232D,20.67,1,16592,Set/3 Decoupage Stacking Tins,United Kingdom,Elizabeth Wong,2023-05-03,42.63,f3c6
536290,536592,2023-05-03,90124B,20.67,1,16592,Blue Murano Twist Necklace,United Kingdom,Elizabeth Wong,2023-05-03,42.63,f3c6


In [39]:
## Check the number of rows of ending with Capital Letter
end_with_cap = final_df[final_df['ProductNo'].str.match(r"^\d{5}[A-Z]$")]
end_with_cap

# Other format = 50380
# End-with-Capital-letter-format = 50380

,TransactionNo,Date,ProductNo,Price,Quantity,CustomerNo,ProductName,Country,Name,date,gbp_thb,id
68,581476,2024-05-10,84596F,10.68,32,12433,Small Marshmallows Pink Bowl,Norway,Kelli Sandoval,2024-05-10,45.77,a19b
69,581476,2024-05-10,84596B,10.68,16,12433,Small Dolly Mix Design Orange Bowl,Norway,Kelli Sandoval,2024-05-10,45.77,a19b
70,581476,2024-05-10,84510A,11.53,20,12433,Set Of 4 English Rose Coasters,Norway,Kelli Sandoval,2024-05-10,45.77,a19b
74,581476,2024-05-10,47559B,11.53,10,12433,Tea Time Oven Glove,Norway,Kelli Sandoval,2024-05-10,45.77,a19b
75,581476,2024-05-10,47504H,11.06,36,12433,English Rose Spirit Level,Norway,Kelli Sandoval,2024-05-10,45.77,a19b
...,...,...,...,...,...,...,...,...,...,...,...,...
536272,536592,2023-05-03,47590B,21.55,1,16592,Pink Happy Birthday Bunting,United Kingdom,Elizabeth Wong,2023-05-03,42.63,f3c6
536279,536401,2023-05-03,47570B,21.17,1,15862,Tea Time Table Cloth,United Kingdom,Jonathan Lee,2023-05-03,42.63,f3c6
536289,536592,2023-05-03,85232D,20.67,1,16592,Set/3 Decoupage Stacking Tins,United Kingdom,Elizabeth Wong,2023-05-03,42.63,f3c6
536290,536592,2023-05-03,90124B,20.67,1,16592,Blue Murano Twist Necklace,United Kingdom,Elizabeth Wong,2023-05-03,42.63,f3c6


In [40]:
# Look into ProductName column. We can see that capital letter defines the color
final_df[final_df['ProductNo'].str.match(r"^\d{5}[A-Z]$")].head(50)

,TransactionNo,Date,ProductNo,Price,Quantity,CustomerNo,ProductName,Country,Name,date,gbp_thb,id
68,581476,2024-05-10,84596F,10.68,32,12433,Small Marshmallows Pink Bowl,Norway,Kelli Sandoval,2024-05-10,45.77,a19b
69,581476,2024-05-10,84596B,10.68,16,12433,Small Dolly Mix Design Orange Bowl,Norway,Kelli Sandoval,2024-05-10,45.77,a19b
70,581476,2024-05-10,84510A,11.53,20,12433,Set Of 4 English Rose Coasters,Norway,Kelli Sandoval,2024-05-10,45.77,a19b
74,581476,2024-05-10,47559B,11.53,10,12433,Tea Time Oven Glove,Norway,Kelli Sandoval,2024-05-10,45.77,a19b
75,581476,2024-05-10,47504H,11.06,36,12433,English Rose Spirit Level,Norway,Kelli Sandoval,2024-05-10,45.77,a19b
90,581477,2024-05-10,84970L,11.53,12,13426,Single Heart Zinc T-Light Holder,United Kingdom,Dalton Graves,2024-05-10,45.77,a19b
98,581478,2024-05-10,84997C,14.50,4,17364,Childrens Cutlery Polkadot Blue,United Kingdom,Michelle James,2024-05-10,45.77,a19b
99,581478,2024-05-10,84997D,14.50,4,17364,Childrens Cutlery Polkadot Pink,United Kingdom,Michelle James,2024-05-10,45.77,a19b
115,581480,2024-05-10,84029E,14.61,8,14441,Red Woolly Hottie White Heart,United Kingdom,Thomas Hull,2024-05-10,45.77,a19b
118,581480,2024-05-10,84029G,14.61,12,14441,Knitted Union Flag Hot Water Bottle,United Kingdom,Thomas Hull,2024-05-10,45.77,a19b


In [41]:
countries = []
for country in final_df['Country']:
    if country not in countries:
        countries.append(country)

print(countries, end='')

['United Kingdom', 'Norway', 'Belgium', 'Germany', 'France', 'Austria', 'Netherlands', 'EIRE', 'USA', 'Channel Islands', 'Iceland', 'Portugal', 'Spain', 'Finland', 'Italy', 'Greece', 'Japan', 'Sweden', 'Denmark', 'Cyprus', 'Malta', 'Switzerland', 'Australia', 'Czech Republic', 'Poland', 'Hong Kong', 'Singapore', 'RSA', 'Israel', 'Unspecified', 'United Arab Emirates', 'Canada', 'European Community', 'Bahrain', 'Brazil', 'Saudi Arabia', 'Lebanon', 'Lithuania']

In [42]:
# Found out country named Unspecified
final_df[final_df['Country'] == 'Unspecified']

,TransactionNo,Date,ProductNo,Price,Quantity,CustomerNo,ProductName,Country,Name,date,gbp_thb,id
216838,565303,2024-02-02,22904,13.27,1,17303,Calendar Paper Cut Design,Unspecified,Timothy Long,2024-02-02,44.772,c79c
216839,565303,2024-02-02,21329,11.94,1,17303,Dinosaurs Writing Set,Unspecified,Timothy Long,2024-02-02,44.772,c79c
216840,565303,2024-02-02,21992,11.53,1,17303,Vintage Paisley Stationery Set,Unspecified,Timothy Long,2024-02-02,44.772,c79c
216841,565303,2024-02-02,20772,12.86,1,17303,Garden Path Journal,Unspecified,Timothy Long,2024-02-02,44.772,c79c
216842,565303,2024-02-02,23196,11.74,1,17303,Vintage Leaf Magnetic Notepad,Unspecified,Timothy Long,2024-02-02,44.772,c79c
...,...,...,...,...,...,...,...,...,...,...,...,...
509114,538635,2023-05-15,22584,12.86,2,17303,Pack Of 6 Panettone Gift Boxes,Unspecified,Timothy Long,2023-05-15,42.329,934b
509115,538635,2023-05-15,22837,15.02,1,17303,Hot Water Bottle Babushka,Unspecified,Timothy Long,2023-05-15,42.329,934b
509116,538635,2023-05-15,22469,11.94,3,17303,Heart Of Wicker Small,Unspecified,Timothy Long,2023-05-15,42.329,934b
509117,538635,2023-05-15,22191,18.96,1,17303,Ivory Diner Wall Clock,Unspecified,Timothy Long,2023-05-15,42.329,934b


In [43]:
# Change Unspecified to Thailand
final_df['Country'] = final_df['Country'].apply(lambda x:'Thailand' if x == 'Unspecified' else x)
final_df[final_df['Country'] == 'Thailand']

/var/folders/0_/s8sxzn9n5jq5cx7f64dbfmj40000gn/T/ipykernel_3383/493487127.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  final_df['Country'] = final_df['Country'].apply(lambda x:'Thailand' if x == 'Unspecified' else x)


,TransactionNo,Date,ProductNo,Price,Quantity,CustomerNo,ProductName,Country,Name,date,gbp_thb,id
216838,565303,2024-02-02,22904,13.27,1,17303,Calendar Paper Cut Design,Thailand,Timothy Long,2024-02-02,44.772,c79c
216839,565303,2024-02-02,21329,11.94,1,17303,Dinosaurs Writing Set,Thailand,Timothy Long,2024-02-02,44.772,c79c
216840,565303,2024-02-02,21992,11.53,1,17303,Vintage Paisley Stationery Set,Thailand,Timothy Long,2024-02-02,44.772,c79c
216841,565303,2024-02-02,20772,12.86,1,17303,Garden Path Journal,Thailand,Timothy Long,2024-02-02,44.772,c79c
216842,565303,2024-02-02,23196,11.74,1,17303,Vintage Leaf Magnetic Notepad,Thailand,Timothy Long,2024-02-02,44.772,c79c
...,...,...,...,...,...,...,...,...,...,...,...,...
509114,538635,2023-05-15,22584,12.86,2,17303,Pack Of 6 Panettone Gift Boxes,Thailand,Timothy Long,2023-05-15,42.329,934b
509115,538635,2023-05-15,22837,15.02,1,17303,Hot Water Bottle Babushka,Thailand,Timothy Long,2023-05-15,42.329,934b
509116,538635,2023-05-15,22469,11.94,3,17303,Heart Of Wicker Small,Thailand,Timothy Long,2023-05-15,42.329,934b
509117,538635,2023-05-15,22191,18.96,1,17303,Ivory Diner Wall Clock,Thailand,Timothy Long,2023-05-15,42.329,934b


### Calculate total_price in THB

In [44]:
# Convert Price in GBP to THB
final_df['Price'] = (final_df['Price'] * final_df['gbp_thb']).round(2)
final_df

/var/folders/0_/s8sxzn9n5jq5cx7f64dbfmj40000gn/T/ipykernel_3383/3261680963.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  final_df['Price'] = (final_df['Price'] * final_df['gbp_thb']).round(2)


,TransactionNo,Date,ProductNo,Price,Quantity,CustomerNo,ProductName,Country,Name,date,gbp_thb,id
0,581482,2024-05-10,22485,982.68,12,17490,Set Of 2 Wooden Market Crates,United Kingdom,Sara Griffin,2024-05-10,45.77,a19b
1,581475,2024-05-10,22596,487.45,36,13069,Christmas Star Wish List Chalkboard,United Kingdom,Michael Holt,2024-05-10,45.77,a19b
2,581475,2024-05-10,23235,527.73,12,13069,Storage Tin Vintage Leaf,United Kingdom,Michael Holt,2024-05-10,45.77,a19b
3,581475,2024-05-10,23272,487.45,12,13069,Tree T-Light Holder Willie Winkie,United Kingdom,Michael Holt,2024-05-10,45.77,a19b
4,581475,2024-05-10,23239,546.49,6,13069,Set Of 4 Knick Knack Tins Poppies,United Kingdom,Michael Holt,2024-05-10,45.77,a19b
...,...,...,...,...,...,...,...,...,...,...,...,...
536345,C536548,2023-05-03,22168,808.26,-2,12472,Organiser Wood Antique White,Germany,Stephen Pena,2023-05-03,42.63,f3c6
536346,C536548,2023-05-03,21218,600.66,-3,12472,Red Spotty Biscuit Tin,Germany,Stephen Pena,2023-05-03,42.63,f3c6
536347,C536548,2023-05-03,20957,500.48,-1,12472,Porcelain Hanging Bell Small,Germany,Stephen Pena,2023-05-03,42.63,f3c6
536348,C536548,2023-05-03,22580,697.00,-4,12472,Advent Calendar Gingham Sack,Germany,Stephen Pena,2023-05-03,42.63,f3c6


In [45]:
# Total sales in THB
final_df['total_amount'] = final_df['Price'] * final_df['Quantity']
final_df

/var/folders/0_/s8sxzn9n5jq5cx7f64dbfmj40000gn/T/ipykernel_3383/1174658905.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  final_df['total_amount'] = final_df['Price'] * final_df['Quantity']


,TransactionNo,Date,ProductNo,Price,Quantity,CustomerNo,ProductName,Country,Name,date,gbp_thb,id,total_amount
0,581482,2024-05-10,22485,982.68,12,17490,Set Of 2 Wooden Market Crates,United Kingdom,Sara Griffin,2024-05-10,45.77,a19b,11792.16
1,581475,2024-05-10,22596,487.45,36,13069,Christmas Star Wish List Chalkboard,United Kingdom,Michael Holt,2024-05-10,45.77,a19b,17548.20
2,581475,2024-05-10,23235,527.73,12,13069,Storage Tin Vintage Leaf,United Kingdom,Michael Holt,2024-05-10,45.77,a19b,6332.76
3,581475,2024-05-10,23272,487.45,12,13069,Tree T-Light Holder Willie Winkie,United Kingdom,Michael Holt,2024-05-10,45.77,a19b,5849.40
4,581475,2024-05-10,23239,546.49,6,13069,Set Of 4 Knick Knack Tins Poppies,United Kingdom,Michael Holt,2024-05-10,45.77,a19b,3278.94
...,...,...,...,...,...,...,...,...,...,...,...,...,...
536345,C536548,2023-05-03,22168,808.26,-2,12472,Organiser Wood Antique White,Germany,Stephen Pena,2023-05-03,42.63,f3c6,-1616.52
536346,C536548,2023-05-03,21218,600.66,-3,12472,Red Spotty Biscuit Tin,Germany,Stephen Pena,2023-05-03,42.63,f3c6,-1801.98
536347,C536548,2023-05-03,20957,500.48,-1,12472,Porcelain Hanging Bell Small,Germany,Stephen Pena,2023-05-03,42.63,f3c6,-500.48
536348,C536548,2023-05-03,22580,697.00,-4,12472,Advent Calendar Gingham Sack,Germany,Stephen Pena,2023-05-03,42.63,f3c6,-2788.00


### Miscellaneous

In [46]:
# Drop unused columns
final_df = final_df.drop(columns=['date', 'gbp_thb', 'id'], axis=1)
final_df

,TransactionNo,Date,ProductNo,Price,Quantity,CustomerNo,ProductName,Country,Name,total_amount
0,581482,2024-05-10,22485,982.68,12,17490,Set Of 2 Wooden Market Crates,United Kingdom,Sara Griffin,11792.16
1,581475,2024-05-10,22596,487.45,36,13069,Christmas Star Wish List Chalkboard,United Kingdom,Michael Holt,17548.20
2,581475,2024-05-10,23235,527.73,12,13069,Storage Tin Vintage Leaf,United Kingdom,Michael Holt,6332.76
3,581475,2024-05-10,23272,487.45,12,13069,Tree T-Light Holder Willie Winkie,United Kingdom,Michael Holt,5849.40
4,581475,2024-05-10,23239,546.49,6,13069,Set Of 4 Knick Knack Tins Poppies,United Kingdom,Michael Holt,3278.94
...,...,...,...,...,...,...,...,...,...,...
536345,C536548,2023-05-03,22168,808.26,-2,12472,Organiser Wood Antique White,Germany,Stephen Pena,-1616.52
536346,C536548,2023-05-03,21218,600.66,-3,12472,Red Spotty Biscuit Tin,Germany,Stephen Pena,-1801.98
536347,C536548,2023-05-03,20957,500.48,-1,12472,Porcelain Hanging Bell Small,Germany,Stephen Pena,-500.48
536348,C536548,2023-05-03,22580,697.00,-4,12472,Advent Calendar Gingham Sack,Germany,Stephen Pena,-2788.00


In [47]:
# Rename columns
final_df = final_df.rename(
    columns={
        'TransactionNo':'transaction_id',
        'Date':'date',
        'ProductNo':'product_id',
        'Price':'price',
        'Quantity':'quantity',
        'CustomerNo':'customer_id',
        'ProductName':'product_name',
        'Country':'country',
        'Name':'customer_name'
    }
)

final_df

,transaction_id,date,product_id,price,quantity,customer_id,product_name,country,customer_name,total_amount
0,581482,2024-05-10,22485,982.68,12,17490,Set Of 2 Wooden Market Crates,United Kingdom,Sara Griffin,11792.16
1,581475,2024-05-10,22596,487.45,36,13069,Christmas Star Wish List Chalkboard,United Kingdom,Michael Holt,17548.20
2,581475,2024-05-10,23235,527.73,12,13069,Storage Tin Vintage Leaf,United Kingdom,Michael Holt,6332.76
3,581475,2024-05-10,23272,487.45,12,13069,Tree T-Light Holder Willie Winkie,United Kingdom,Michael Holt,5849.40
4,581475,2024-05-10,23239,546.49,6,13069,Set Of 4 Knick Knack Tins Poppies,United Kingdom,Michael Holt,3278.94
...,...,...,...,...,...,...,...,...,...,...
536345,C536548,2023-05-03,22168,808.26,-2,12472,Organiser Wood Antique White,Germany,Stephen Pena,-1616.52
536346,C536548,2023-05-03,21218,600.66,-3,12472,Red Spotty Biscuit Tin,Germany,Stephen Pena,-1801.98
536347,C536548,2023-05-03,20957,500.48,-1,12472,Porcelain Hanging Bell Small,Germany,Stephen Pena,-500.48
536348,C536548,2023-05-03,22580,697.00,-4,12472,Advent Calendar Gingham Sack,Germany,Stephen Pena,-2788.00


In [48]:
final_df.dtypes

transaction_id            object
date              datetime64[ns]
product_id                object
price                    float64
quantity                   int64
customer_id                int64
product_name              object
country                   object
customer_name             object
total_amount             float64
dtype: object